In [1]:
import sys
import os

# Get the absolute path of the parent directory.
parent_dir = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))

# Add the parent directory to the system path to be able to import modules from 'lib.'
sys.path.append(parent_dir)

In [2]:
%%capture
import datasets

import ipywidgets as widgets
from IPython.display import HTML, Markdown as md
import itertools

from lib.memory import DSDM
from lib.utils import cleanup, configs, inference, learning, utils 

import math
import numpy as np
import random

import pandas as pd
import pathlib
import pickle

import string


### Package options ###
pd.set_option('display.max_rows', 500)

In [3]:
# Set seed.
utils.fix_seed(41)

Using seed: 41

In [4]:
### Utils ###
def get_results(
    sentences: list[str],
    retrieve_mode: str,
    remove_stopwords_inference: bool = True
) -> None:
    
    # Retrieve content from memory.
    retrieved_contents = inference.infer(
        memory.address_size,
        cleanup,
        memory,
        sentences,
        retrieve_mode=retrieve_mode,
        remove_stopwords=remove_stopwords_inference,
        k=7, 
    )
    
    if retrieve_mode == "top_k":
        sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity']) 
    
        for s, addresses in zip(sentences, retrieved_contents):
            display(md(f"<ins>**Sentence:**</ins> {s}"))
#             out_tables = []
            display(md("<ins>**Addresses:**</ins>"))
            for a in addresses:
                address_sims_df = inference.get_similarities_to_atomic_set(
                    a, cleanup, k=11)
#                 out = widgets.Output()
#                 with out:
#                     display(address_sims_df)
#                 out_tables.append(out)
#             display(widgets.HBox(out_tables))
                display(address_sims_df)
        return
    elif retrieve_mode == "pooling":  
        sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity']) 

        for s, c in zip(sentences, retrieved_contents):
            sentence_sims_df = inference.get_similarities_to_atomic_set(
                c, cleanup)
            sentence_sims_df['sentence'] = [s] * len(sentence_sims_df)
            sims_df = pd.concat([sims_df, sentence_sims_df])

        sims_df = sims_df.sort_values(['sentence', 'similarity'], ascending=False) \
                         .set_index(['sentence', 'token'])

        display(sims_df)
        return
    else:  # unrecognized
        display(md("Unrecognized retrieval mode."))
        return

In [5]:
in_sentences = [
    """Blaine was reared in a Prohibition home, and while still a young girl, she became a very active participant at temperance meetings, where she won great favor for her songs and recitations.""",
    """In 1910, she was elected to the position of organizer and lecturer of the National WCTU.""",
    """Another feature of her work was the organization of temperance mass-meetings of Sunday-school children, usually preceded by a formal parade.""",
    """With all other games played, a victory over Everton had put United top of the group on nine points.""",
    """The 2022 FA Women's League Cup Final was the 11th final of the FA Women's League Cup, England's secondary cup competition for women's football teams and its primary league cup tournament.""",
    """In 2020 Mico's single 'igare' awarded as the best song of the summer in Kiss Summer Awards.""",
    """She collected the speech and words of Dublin city and donated her collection to the Department of Irish Folklore at University College, Dublin.""",
    """Traditional palyanytsya was baked from yeast dough.""",
   """First, hops were boiled in a pot, which was then poured into a makitra, to which sifted wheat flour was added.""",
     """ Jonathan Holland of ScreenDaily deemed the film to be "superbly directed by Palomero, who seems to have a special gift for seeing the world through children's eyes." """   
]

In [6]:
out_sentences = [
    """As the population of all of the towns grew, the need for better transportation between them also grew.""",
    """The construction of the line was the subject of a legal challenge.""",
    """The extremes, which can be fatal to healthy people within six hours, could affect hundreds of millions of people unused to such conditions.""",
    """Dangerous heat stress events will spread rapidly across the world as global heating continues.""",
    """Whether or not history will determine that we are living in an ever more divided culture, it certainly feels that way."""
]

In [22]:
# Parametrized cell.
filename = "2023-09-10 17-02-12-946618.pkl"
experiment_no = 3

In [23]:
# Load memory and associated cleanup.
cleanup = pickle.load(open(f"cleanups/method2/cleanup_{filename}",'rb'))
memory = pickle.load(open(f"memories/method2/memory_{filename}",'rb'))

In [24]:
display(md(f"# Mining Transfomer attention - Experiment {experiment_no}"))

# Mining Transfomer attention - Experiment 3

## Memory visualiztion
### Statistics

In [9]:
display(md(f"Number of trained articles: {len(memory.wiki_articles)}"))
display(md(f"Number of memory updates: {memory.n_updates}"))
display(md(f"Number of memory expansions: {memory.n_expansions}"))
display(md(f"Updates percentage: {round(memory.n_updates / (memory.n_updates + memory.n_expansions), 3)}%"))
display(md(f"Number of existing memory addresses: {len(memory.addresses)}"))

Number of trained articles: 12

Number of memory updates: 1673

Number of memory expansions: 3159

Updates percentage: 0.346%

Number of existing memory addresses: 3159

### Addresses

In [ ]:
display(md("Visualize 30 randomly sampled addresses."))

In [10]:
addresses = np.random.randint(0, len(memory.addresses), size=30)

for address in addresses:
    display(md(f"### <ins>Address {address}</ins>"))
    display(md(f"Address **chunk score:** {memory.scores[address][0]}, **bin score:** {memory.scores[address][1]}"))
    address_sims_df = inference.get_similarities_to_atomic_set(
            memory.addresses[address],
            cleanup,
    )
    display(address_sims_df)

### <ins>Address 1984</ins>

Address **chunk score:** 0.57, **bin score:** 0.2349659478641115

,token,similarity
0,last,0.51
1,as,0.51
2,s,0.49
3,game,0.47
4,removed,0.11
5,feature,0.10
6,mm,0.09
7,intention,0.08
8,beach,0.08
9,1932,0.08


### <ins>Address 931</ins>

Address **chunk score:** 1.0, **bin score:** 0.7685002927901223

...

Address **chunk score:** 0.65, **bin score:** 0.3213849641615525

,token,similarity
0,2004,0.68
1,births,0.68
2,facilities,0.11
3,opening,0.10
4,shovel,0.09
5,encircled,0.09
6,children,0.09
7,grown,0.08
8,competed,0.08
9,similar,0.08


### <ins>Address 2749</ins>

Address **chunk score:** 0.88, **bin score:** 0.08983558841282502

,token,similarity
0,and,0.71
1,shooting,0.71
2,can,0.14
3,finding,0.11
4,sifted,0.09
5,again,0.09
6,well,0.08
7,commission,0.08
8,union,0.08
9,numerous,0.08


## In-sample sentences
### W/ stop words in inference sentence
#### Closest addresses

In [11]:
get_results(in_sentences[:3], "top_k", False)

<ins>**Sentence:**</ins> Blaine was reared in a Prohibition home, and while still a young girl, she became a very active participant at temperance meetings, where she won great favor for her songs and recitations.

<ins>**Addresses:**</ins>

,token,similarity
0,she,0.39
1,participant,0.36
2,at,0.36
3,meetings,0.35
4,temperance,0.32
5,active,0.32
6,became,0.31
7,a,0.31
8,very,0.30
9,outer,0.11


,token,similarity
0,a,0.47
1,and,0.45
2,still,0.44
3,while,0.43
4,young,0.43
5,february,0.11
6,3rd,0.10
7,during,0.10
8,when,0.09
9,la,0.09


,token,similarity
0,girl,0.42
1,a,0.41
2,still,0.41
3,and,0.40
4,while,0.40
5,young,0.40
6,february,0.11
7,cutting,0.10
8,during,0.09
9,pictures,0.09


...

,token,similarity
0,her,0.45
1,another,0.45
2,work,0.44
3,feature,0.41
4,of,0.37
5,was,0.35
6,off,0.11
7,name,0.10
8,rtve,0.09
9,bottom,0.09


,token,similarity
0,children,0.48
1,of,0.44
2,meetings,0.43
3,school,0.43
4,sunday,0.43
5,circular,0.09
6,unusual,0.09
7,pregnant,0.08
8,fell,0.08
9,uncle,0.08


,token,similarity
0,of,0.89
1,the,0.45
2,mary,0.10
3,brazier,0.08
4,roof,0.08
5,halfway,0.08
6,mothers,0.08
7,anti,0.08
8,living,0.08
9,convex,0.08


,token,similarity
0,meetings,0.52
1,of,0.51
2,children,0.50
3,organization,0.48
4,park,0.08
5,both,0.08
6,member,0.08
7,surrendered,0.08
8,t,0.08
9,mills,0.07


,token,similarity
0,the,0.51
1,organization,0.51
2,temperance,0.50
3,of,0.49
4,convex,0.11
5,roof,0.09
6,appearance,0.09
7,relation,0.08
8,snail,0.08
9,matches,0.08


#### Pooled address space

In [12]:
get_results(in_sentences[:3], "pooling", False)

similarity
sentence                                           token              
In 1910, she was elected to the position of org... the            0.76
                                                   of             0.37
                                                   in             0.32
                                                   and            0.25
                                                   to             0.21
                                                   usually        0.21
                                                   a              0.18
                                                   was            0.14
                                                   with           0.13
                                                   bruno          0.11
Blaine was reared in a Prohibition home, and wh... the            0.62
                                                   a              0.38
                                                   and            0.35
                                                   in             0.35
                                                   of             0.29
                                                   usually        0.23
                                                   to             0.23
                                                   was            0.16
                                                   with           0.15
                                                   at             0.11
Another feature of her work was the organizatio... the            0.70
                                                   of             0.43
                                                   in             0.28
                                                   a              0.27
                                                   and            0.23
                                                   usually        0.22
                                                   to             0.22
                                                   was            0.15
                                                   with           0.14
                                                   bruno          0.11

### W/o stop words in inference sentence

In [13]:
get_results(in_sentences[:3], "top_k", True)

<ins>**Sentence:**</ins> Blaine was reared in a Prohibition home, and while still a young girl, she became a very active participant at temperance meetings, where she won great favor for her songs and recitations.

<ins>**Addresses:**</ins>

,token,similarity
0,she,0.39
1,participant,0.36
2,at,0.36
3,meetings,0.35
4,temperance,0.32
5,active,0.32
6,became,0.31
7,a,0.31
8,very,0.30
9,outer,0.11


,token,similarity
0,blaine,0.72
1,prohibition,0.72
2,ovid,0.10
3,turn,0.09
4,christian,0.09
5,born,0.09
6,tracks,0.09
7,rules,0.09
8,dough,0.08
9,watford,0.08


,token,similarity
0,prohibition,0.60
1,was,0.60
2,reared,0.56
3,ovid,0.12
4,125,0.10
5,two,0.10
6,connected,0.10
7,if,0.10
8,i,0.09
9,would,0.09


...

,token,similarity
0,of,0.54
1,another,0.31
2,mass,0.31
3,organization,0.29
4,work,0.29
5,the,0.29
6,her,0.27
7,temperance,0.27
8,feature,0.27
9,was,0.26


,token,similarity
0,meetings,0.52
1,of,0.51
2,children,0.50
3,organization,0.48
4,park,0.08
5,both,0.08
6,member,0.08
7,surrendered,0.08
8,t,0.08
9,mills,0.07


,token,similarity
0,school,0.70
1,children,0.70
2,surrendered,0.09
3,gone,0.09
4,team,0.08
5,ranking,0.08
6,her,0.08
7,facilities,0.08
8,often,0.08
9,specialise,0.08


,token,similarity
0,her,0.45
1,another,0.45
2,work,0.44
3,feature,0.41
4,of,0.37
5,was,0.35
6,off,0.11
7,name,0.10
8,rtve,0.09
9,bottom,0.09


,token,similarity
0,usually,1.00
1,to,1.00
2,meetings,0.12
3,temperance,0.09
4,opened,0.09
5,significant,0.09
6,bid,0.09
7,cut,0.08
8,lives,0.08
9,park,0.08


In [14]:
get_results(in_sentences[:3], "pooling", True)

similarity
sentence                                           token              
In 1910, she was elected to the position of org... the            0.69
                                                   in             0.34
                                                   of             0.33
                                                   and            0.29
                                                   a              0.26
                                                   usually        0.23
                                                   to             0.23
                                                   with           0.15
                                                   was            0.15
                                                   bruno          0.11
Blaine was reared in a Prohibition home, and wh... the            0.70
                                                   in             0.33
                                                   of             0.30
                                                   and            0.28
                                                   to             0.26
                                                   usually        0.26
                                                   a              0.25
                                                   with           0.15
                                                   was            0.14
                                                   bruno          0.11
Another feature of her work was the organizatio... the            0.68
                                                   in             0.33
                                                   to             0.32
                                                   usually        0.32
                                                   of             0.29
                                                   and            0.27
                                                   a              0.25
                                                   with           0.16
                                                   was            0.14
                                                   bruno          0.11

## Out-of-sample sentences
### W/ stop words in inference sentence
#### Closest addresses

In [15]:
get_results(out_sentences[:3], "top_k", False)

<ins>**Sentence:**</ins> As the population of all of the towns grew, the need for better transportation between them also grew.

<ins>**Addresses:**</ins>

,token,similarity
0,the,0.71
1,of,0.71
2,better,0.71
3,transportation,0.71
4,convex,0.09
5,fa,0.09
6,living,0.09
7,mary,0.08
8,roof,0.08
9,anti,0.08


,token,similarity
0,the,0.81
1,transportation,0.81
2,better,0.43
3,of,0.43
4,evolution,0.41
5,convex,0.10
6,fa,0.09
7,sliced,0.09
8,roof,0.08
9,living,0.08


,token,similarity
0,the,0.82
1,transportation,0.82
2,secretary,0.42
3,better,0.41
4,of,0.41
5,fa,0.10
6,11th,0.09
7,january,0.08
8,off,0.08
9,sliced,0.08


...

,token,similarity
0,better,0.48
1,of,0.48
2,municipality,0.47
3,affect,0.47
4,a,0.45
5,fatal,0.45
6,transportation,0.45
7,the,0.45
8,could,0.43
9,part,0.43


,token,similarity
0,better,0.53
1,of,0.53
2,municipality,0.51
3,affect,0.51
4,a,0.51
5,fatal,0.51
6,could,0.49
7,part,0.49
8,which,0.10
9,identify,0.09


,token,similarity
0,of,0.58
1,better,0.58
2,descent,0.57
3,people,0.55
4,whitish,0.10
5,varies,0.10
6,last,0.10
7,position,0.09
8,local,0.09
9,bottom,0.08


,token,similarity
0,legal,0.37
1,in,0.37
2,province,0.37
3,six,0.37
4,the,0.34
5,transportation,0.34
6,healthy,0.34
7,village,0.34
8,hours,0.34
9,groningen,0.34


,token,similarity
0,municipality,0.54
1,affect,0.54
2,of,0.51
3,better,0.51
4,the,0.51
5,transportation,0.51
6,could,0.48
7,part,0.48
8,convex,0.09
9,2011,0.08


#### Pooled address space

In [16]:
get_results(out_sentences[:3], "pooling", False)

similarity
sentence                                           token                     
The extremes, which can be fatal to healthy peo... transportation        0.68
                                                   the                   0.68
                                                   of                    0.37
                                                   better                0.37
                                                   six                   0.34
                                                   in                    0.34
                                                   to                    0.28
                                                   usually               0.28
                                                   a                     0.26
                                                   fatal                 0.26
The construction of the line was the subject of... transportation        0.82
                                                   the                   0.82
                                                   of                    0.36
                                                   better                0.36
                                                   in                    0.24
                                                   six                   0.24
                                                   fatal                 0.20
                                                   a                     0.20
                                                   and                   0.19
                                                   to                    0.15
As the population of all of the towns grew, the... transportation        0.83
                                                   the                   0.83
                                                   of                    0.36
                                                   better                0.36
                                                   in                    0.23
                                                   six                   0.23
                                                   and                   0.18
                                                   a                     0.16
                                                   fatal                 0.16
                                                   to                    0.15

### W/o stop words in inference sentence
#### Closest addresses

In [17]:
get_results(out_sentences[:3], "top_k", True)

<ins>**Sentence:**</ins> As the population of all of the towns grew, the need for better transportation between them also grew.

<ins>**Addresses:**</ins>

,token,similarity
0,towns,0.70
1,an,0.70
2,derivative,0.70
3,grew,0.70
4,capitol,0.10
5,so,0.09
6,straight,0.09
7,chelsea,0.09
8,riding,0.09
9,run,0.08


,token,similarity
0,formal,0.60
1,8,0.60
2,derivative,0.56
3,grew,0.56
4,population,0.55
5,car,0.55
6,brown,0.09
7,capitol,0.09
8,creative,0.09
9,went,0.09


,token,similarity
0,the,0.71
1,of,0.71
2,better,0.71
3,transportation,0.71
4,convex,0.09
5,fa,0.09
6,living,0.09
7,mary,0.08
8,roof,0.08
9,anti,0.08


...

,token,similarity
0,extremes,0.59
1,is,0.59
2,part,0.58
3,could,0.58
4,affect,0.58
5,municipality,0.58
6,italian,0.10
7,receiving,0.10
8,folklore,0.09
9,retaining,0.09


,token,similarity
0,a,0.58
1,fatal,0.58
2,healthy,0.57
3,village,0.57
4,extremes,0.56
5,is,0.56
6,hearth,0.10
7,production,0.10
8,rehearsals,0.09
9,outside,0.09


,token,similarity
0,municipality,0.58
1,affect,0.58
2,is,0.58
3,extremes,0.58
4,fatal,0.57
5,a,0.57
6,identify,0.10
7,standardized,0.09
8,carried,0.09
9,icaa,0.08


,token,similarity
0,part,0.59
1,could,0.59
2,is,0.58
3,extremes,0.58
4,fatal,0.57
5,a,0.57
6,folklore,0.10
7,not,0.10
8,outright,0.09
9,italian,0.09


,token,similarity
0,legal,0.37
1,in,0.37
2,province,0.37
3,six,0.37
4,the,0.34
5,transportation,0.34
6,healthy,0.34
7,village,0.34
8,hours,0.34
9,groningen,0.34


#### Pooled address space

In [18]:
get_results(out_sentences[:3], "pooling", True)

similarity
sentence                                           token                     
The extremes, which can be fatal to healthy peo... transportation        0.65
                                                   the                   0.65
                                                   in                    0.39
                                                   six                   0.39
                                                   a                     0.32
                                                   fatal                 0.32
                                                   better                0.31
                                                   of                    0.31
                                                   and                   0.29
                                                   to                    0.24
The construction of the line was the subject of... transportation        0.68
                                                   the                   0.68
                                                   in                    0.36
                                                   six                   0.36
                                                   of                    0.31
                                                   better                0.31
                                                   and                   0.28
                                                   a                     0.26
                                                   fatal                 0.26
                                                   to                    0.25
As the population of all of the towns grew, the... transportation        0.79
                                                   the                   0.79
                                                   of                    0.35
                                                   better                0.35
                                                   in                    0.27
                                                   six                   0.27
                                                   and                   0.22
                                                   a                     0.20
                                                   fatal                 0.20
                                                   to                    0.18

In [19]:
# # Set retrieve mode.
# retrieve_mode = "top_k"

# # Get table with token similarities for each "out-of-train" sentence.
# retrieved_contents = inference.infer(
#     memory.address_size,
#     cleanup,
#     memory,
#     sentences,
#     retrieve_mode=retrieve_mode,
#     remove_stopwords=True,
#     k=7, 
# )


# sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity']) 

# for s, addresses in zip(sentences, retrieved_contents):
#     display(s)
#     out_tables = []
#     for a in addresses:
#         address_sims_df = inference.get_similarities_to_atomic_set(
#             a, cleanup, k=11)
#         out = widgets.Output()
#         with out:
#             display(address_sims_df)
#         out_tables.append(out)
#     display(widgets.HBox(out_tables))

In [20]:
# retrieve_mode = "pooling"

# # Get table with token similarities for each "out-of-train" sentence.
# retrieved_contents = inference.infer(
#     memory.address_size,
#     cleanup,
#     memory,
#     sentences,
#     retrieve_mode=retrieve_mode,
#     remove_stopwords=True,
#     k=7, 
# )

# sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity']) 
      
# for s, c in zip(sentences, retrieved_contents):
#     sentence_sims_df = inference.get_similarities_to_atomic_set(
#         c, cleanup)
#     sentence_sims_df['sentence'] = [s] * len(sentence_sims_df)
#     sims_df = pd.concat([sims_df, sentence_sims_df])

# sims_df = sims_df.sort_values(['sentence', 'similarity'], ascending=False) \
#                  .set_index(['sentence', 'token'])

# display(sims_df)